### 자연어 감성분석
- 감성사전 기반 : 미리정의된 감성 단어 사전 사용(규칙 기반)
    - TextBlob, AFINNm VADER
- 머신러닝 기반 : 데이터로부터 패턴 학습(통계 기반)
    - TF-IDF 벡터화
    - 선형회귀
    - 로지스틱회귀
    - F1 Score, Recison, Recall  -> classification report
### 사용 데이터
- NLTK 영화 리뷰(2000개)
- 다음영화리뷰

#### 알고리즘
 - TextBlob     사전기반 감성분석
 - AFINN        감정 점수 매핑
 - VADER(Valence Aware Dictionary)  소셜미디어 최적화 감성분석
 - TF-IDF       텍스트 벡터화
 - Multinomial Naive Bayes          확률 기반 분류

In [1]:
# TextBlob
# 이 영화는 정말 좋고 재미있다
    # 좋다 +1(긍정)
    # 재미있다 +1(긍정)
    # +2 > 0 --> 긍정(pos) 분류
# Polarity(극성도)  (긍정단어수 개수 - 부정단어 개수) / 전체 단어 개수
# -1.0 ~ +1.0
# 0 중립
# Subjectivity(주관성) 평가대상 단어 비율
# 0.0 ~ 1.0
# 0 : 객관적   1 : 주관적

# 문맥무시하고 단어 극성만 고려
# 이 영화는 나쁘지 않다  -> 나쁘다(-)  않다(-) 로 인식
# 단점 : 빠른 속도, 학습 불필요
# 사용 : 실시간 감성분석,  스트리밍데이터

In [ ]:
%conda install textblob

In [ ]:
from textblob import TextBlob, Word
text = "TextBlob is amazingly simple to use. What a wonderful library for NLP!"
blob =  TextBlob(text)
print(blob.sentences)
print(blob.words)
print(blob.tags)
import nltk
nltk.download('brown')
print(blob.noun_phrases)
print('-'*100)
# 감성분석
blob.sentiment
print(f'polarity : {blob.sentiment.polarity}')
print(f'subjectivity : {blob.sentiment.subjectivity}')

[Sentence("TextBlob is amazingly simple to use."), Sentence("What a wonderful library for NLP!")]
['TextBlob', 'is', 'amazingly', 'simple', 'to', 'use', 'What', 'a', 'wonderful', 'library', 'for', 'NLP']
[('TextBlob', 'NNP'), ('is', 'VBZ'), ('amazingly', 'RB'), ('simple', 'JJ'), ('to', 'TO'), ('use', 'VB'), ('What', 'WP'), ('a', 'DT'), ('wonderful', 'JJ'), ('library', 'NN'), ('for', 'IN'), ('NLP', 'NNP')]
['textblob', 'wonderful library', 'nlp']
----------------------------------------------------------------------------------------------------
polarity : 0.5
subjectivity : 0.6785714285714286


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\playdata2\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [ ]:
# AFINN(Lexicon-Based)  감성사전
# 각 단어의 -5 ~ +5의 점수를 부여하고 합산
# 이 영화는 좋지만 좋지 않은 부분도 있다
    # 좋다 +3  좋다 +3  나쁘다 -3  =  +3 > 0 긍정
# score = sum(word_sentiment_value)
# 분류규칙 score > 0 긍정  score < 0 부정  
# 이모티콘지원
# 강도표현 인식 very , really 등

# 강조 수정자(intensifiers)
    # 매우좋다 = 1.5X(좋다의 점수)

# AFINN vs TextBlob
# AFINN : 더 정확한 점수 매핑
# TextBlob : 더 일반적인 접근

In [ ]:
%pip install afinn

In [4]:
from afinn import Afinn
af = Afinn()
text1 = "TextBlob is amazingly simple to use"
text2 = "What a wonderful library for NLP!"
score1 = af.score(text1)
score2 = af.score(text2)
score1, score2

(0.0, 4.0)

In [ ]:
# VADER  쇼설 미디어 텍스트에 최적화
# 이 영화는 정말정말 훌륭해!!!
# 훌륭하다 (기본) + 0.7   정말정말 (강조)x1.5
# !!! (문장부호강조) x1.2
# 4개의 감정 지수
    # positive 긍정 확률 0 ~ 1
    # nagative 부정 확률
    # neutral 중립 확률
    # compound 종합점수 -1 ~ 1
# score = compound_score / sqrt(compound_score**2 + 0.0625)
# score >=0.05 긍정
# score <=-0.05 부정  
# 그 사이는 중립
# 대소문자 구분  AMAZING amazing 다른 점수
# :) 긍정    :-(  부정


In [5]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\playdata2\AppData\Roaming\nltk_data...


True

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyer = SentimentIntensityAnalyzer()
sentences = [
    "I love this product! It's absolutely amazing 😍",
    "This is the worst movie I've ever seen...",
    "The food was okay, not great but not bad either.",
    "I’m REALLY happy with the results!!!",
    "Not good at all. I’m disappointed.",
]
for s in sentences:
    scores = analyer.polarity_scores(s)
    print(f'문장 : {s}')
    print(f'점수 : {scores}')


문장 : I love this product! It's absolutely amazing 😍
점수 : {'neg': 0.0, 'neu': 0.318, 'pos': 0.682, 'compound': 0.862}
문장 : This is the worst movie I've ever seen...
점수 : {'neg': 0.369, 'neu': 0.631, 'pos': 0.0, 'compound': -0.6249}
문장 : The food was okay, not great but not bad either.
점수 : {'neg': 0.149, 'neu': 0.487, 'pos': 0.364, 'compound': 0.4728}
문장 : I’m REALLY happy with the results!!!
점수 : {'neg': 0.0, 'neu': 0.472, 'pos': 0.528, 'compound': 0.7651}
문장 : Not good at all. I’m disappointed.
점수 : {'neg': 0.579, 'neu': 0.421, 'pos': 0.0, 'compound': -0.6711}
